In [183]:
import pandas as pd
import numpy as np
import joblib

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [184]:
df                = pd.read_csv('data/Данные для задачи.txt',sep=';')

In [185]:
num_cols          = []
bin_cols          = []
cat_cols          = []

for col in df.columns.drop(['DATA_TYPE','POLICY_ID','POLICY_IS_RENEWED']):        
    if df[col].dtype == 'object':
        cat_cols.append(col)
    elif (df[col].unique().shape[0] == 2) & (df[col].dtype != 'object'):
        bin_cols.append(col)
    else:
        num_cols.append(col)  

In [186]:
#Binary columns
df_bin            = df[bin_cols]

In [187]:
#Numerical columns
scaler            = MinMaxScaler()
scaler_obj        = scaler.fit(df[num_cols])
df_scaled         = pd.DataFrame(scaler_obj.transform(df[num_cols]), columns=num_cols)

In [188]:
#Categorial columns
ohe               = OneHotEncoder(handle_unknown = 'ignore')
ohe_obj           = ohe.fit(df[cat_cols])
cat_ohe_cols      = ohe.get_feature_names(cat_cols)
df_ohe            = pd.DataFrame(ohe_obj.transform(df[cat_cols]).toarray(), columns=cat_ohe_cols)

In [189]:
features          = dict({'num_cols':num_cols,'bin_cols':bin_cols,'cat_cols':cat_cols,'cat_ohe_cols':cat_ohe_cols})

In [190]:
df                = pd.concat((df_bin, df_scaled, df_ohe, df[['DATA_TYPE','POLICY_IS_RENEWED']]),axis=1).set_index(np.arange(0,df.shape[0]))

MemoryError: Unable to allocate 1.57 GiB for an array with shape (2178, 96605) and data type float64

In [ ]:
x_cnt_rows        = df[df['DATA_TYPE'] == 'TRAIN'].shape[0]
x_ts_cnt_rows     = df[df['DATA_TYPE'] == 'TEST '].shape[0]
all_cnt_rows      = df.shape[0]
X                 = df[df['DATA_TYPE'] == 'TRAIN'].drop(('POLICY_IS_RENEWED'), axis=1).set_index(np.arange(0,x_cnt_rows))
X_ts              = df[df['DATA_TYPE'] == 'TEST '].drop(('POLICY_IS_RENEWED'), axis=1).set_index(np.arange(x_cnt_rows, all_cnt_rows))

X                 = X.drop(('DATA_TYPE'), axis=1)
X_ts              = X_ts.drop(('DATA_TYPE'), axis=1)
y                 = df[df['DATA_TYPE'] == 'TRAIN']['POLICY_IS_RENEWED']
y_ts              = df[df['DATA_TYPE'] == 'TEST ']['POLICY_IS_RENEWED']
feature_names     = X.columns
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)

In [ ]:
gbt_model         = GradientBoostingClassifier(n_estimators=100, random_state=11)

In [ ]:
%%time
gbt_model.fit(X,y)

In [ ]:
path              = '../deploy/'
objs              = dict({'feats':features,'scale':scaler_obj,'ohe':ohe_obj,'model':gbt_model})
for obj, file in objs.items():
    joblib.dump(file,path + obj + '.pkl')